In [1]:
import cv2
import mediapipe as mp
import numpy as np
import keypoints
# Initialize MediaPipe Face Mesh.
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

# Function to detect landmarks on a given image
def detect_landmarks(image):
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if not results.multi_face_landmarks:
        return None
    landmarks = results.multi_face_landmarks[0]  # Assuming one face
    return landmarks

# Load initial and final images
initial_image = cv2.imread("data/before/1.png")
final_image = cv2.imread("data/after/1.png")
# Detect landmarks
initial_landmarks = detect_landmarks(initial_image)
final_landmarks = detect_landmarks(final_image)

if not initial_landmarks or not final_landmarks:
    print("Could not detect landmarks in one of the images.")


c:\Users\nikol\Documents\projects\makeapp-pro\lab\env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [2]:
# Extract the key landmarks (e.g., eyes, nose) for alignment
def get_landmark_points(landmarks, image):
    h, w, _ = image.shape
    points = []
    # Use specific landmark indices (for eyes and nose)
    key_landmarks = keypoints.EYE_L3 + keypoints.EYE_R3 + keypoints.NOSE_TIP + keypoints.NOSE_BOTTOM + keypoints.CHEEK_LEFT + keypoints.CHEEK_RIGHT
    for lm_index in key_landmarks:
        x = int(landmarks.landmark[lm_index].x * w)
        y = int(landmarks.landmark[lm_index].y * h)
        points.append([x, y])
    return np.array(points, dtype=np.float32)

# Get corresponding points from both images
initial_points = get_landmark_points(initial_landmarks, initial_image)
final_points = get_landmark_points(final_landmarks, final_image)

# Calculate the affine transformation matrix
affine_matrix = cv2.getAffineTransform(initial_points[:3], final_points[:3])

# Apply affine transformation to warp the initial image
aligned_initial_image = cv2.warpAffine(initial_image, affine_matrix, (final_image.shape[1], final_image.shape[0]))

# Show the aligned image
cv2.imshow("Aligned Initial Image", aligned_initial_image)
cv2.imshow("Final Image", final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# Function to compute similarity
def compute_similarity(image1, image2):
    # Convert both images to grayscale for comparison
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Compute absolute difference between images
    difference = cv2.absdiff(gray1, gray2)

    # Sum up the differences
    similarity_score = np.sum(difference)
    return similarity_score

# Compute the similarity between aligned initial and final images
similarity = compute_similarity(final_image, final_image)
print(f"Similarity Score: {similarity}")

def convert_to_hsv(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    return hsv_image

def compute_histogram(image, mask=None):
    # Compute histograms for Hue and Saturation channels (bins=50 for more granularity)
    hsv_image = convert_to_hsv(image)
    hist_hue = cv2.calcHist([hsv_image], [0], mask, [50], [0, 180])  # Hue channel
    hist_saturation = cv2.calcHist([hsv_image], [1], mask, [50], [0, 256])  # Saturation channel

    # Normalize histograms
    cv2.normalize(hist_hue, hist_hue)
    cv2.normalize(hist_saturation, hist_saturation)

    return hist_hue, hist_saturation

Similarity Score: 0


In [ ]:
import cv2
import numpy as np

# Function to create a mask for a given region (based on landmarks)
def create_region_mask(landmarks, image, region_points):
    h, w, _ = image.shape
    mask = np.zeros((h, w), dtype=np.uint8)

    # Get the coordinates for the region
    points = [(int(landmarks.landmark[p].x * w), int(landmarks.landmark[p].y * h)) for p in region_points]
    
    # Fill the polygon that represents the region
    cv2.fillPoly(mask, [np.array(points)], 255)
    
    return mask

# Function to apply the mask to the image and black out the background
def apply_mask(image, mask):
    # Ensure mask is in 3 channels (for RGB image)
    mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # Apply the mask to the image
    masked_image = cv2.bitwise_and(image, mask_rgb)
    return masked_image

# Function to extract specific landmark points from the detected landmarks
def get_landmark_points(landmarks, image, key_indices):
    h, w, _ = image.shape
    points = []
    for idx in key_indices:
        x = int(landmarks.landmark[idx].x * w)
        y = int(landmarks.landmark[idx].y * h)
        points.append([x, y])
    return np.array(points, dtype=np.float32)

In [ ]:
# Create the mask for the lips region
src_mask = create_region_mask(initial_landmarks, initial_image, keypoints.EYE_L3)
tgt_mask = create_region_mask(final_landmarks, final_image, keypoints.EYE_L3)
# Apply the mask to isolate the lips region
src_eye_mask = apply_mask(initial_image, src_mask)
tgt_eye_mask = apply_mask(final_image, tgt_mask)
# Display the masked image

#calculate similarity
similarity = compute_similarity(tgt_eye_mask, src_eye_mask)
print(f"Similarity Score: {similarity}")
src_points = get_landmark_points(initial_landmarks, initial_image, keypoints.EYE_L3)
tgt_points = get_landmark_points(final_landmarks, final_image, keypoints.EYE_L3)
affine_matrix = cv2.getAffineTransform(src_points, tgt_points)
aligned_initial_mask = cv2.warpAffine(src_mask, affine_matrix, (final_image.shape[1], final_image.shape[0]))


cv2.imshow("Aligned Initial Image", aligned_initial_mask)
cv2.imshow("Final Image", final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Similarity Score: 10359730


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\imgwarp.cpp:3631: error: (-215:Assertion failed) src.checkVector(2, CV_32F) == 3 && dst.checkVector(2, CV_32F) == 3 in function 'cv::getAffineTransform'
